In [1]:
import talk.config as con

# environment variable for MOSEK
con.config_configManager()
con.config_matplotlib()

Leveraged Portfolios
---------------------
https://en.wikipedia.org/wiki/130%E2%80%9330_fund

#### Thomas Schmelzer

### A 130/30 Equity Portfolio

- Allocate capital $C=1$. Sell short at most $c = 0.3$ to finance a long position of $1 + c$. 
- Universe of $n$ assets.

\begin{align}\mathbf{x}^{*}=\arg\max_{\mathbf{x} \in \mathbb{R}^n}& \mu^{T}\mathbf{x}\\
\text{s.t. } &\Sigma\,x_i=1\\
             &\Sigma\,\lvert x_i\rvert \leq 1 + 2c\\
             &\sqrt{\mathbf{x}^T\mathbf{C}\mathbf{x}} \leq \sigma_{\max}
\end{align}

### Cholesky decomposition of the covariance matrix:
$$\sqrt{\mathbf{x}^T\mathbf{C}\mathbf{x}}=\sqrt{\mathbf{x}^T\mathbf{G G}^T\mathbf{x}}=\rVert{\mathbf{G}^T\mathbf{x}}\lVert_2$$
Introduce the cone $[y, G^T x] \in \mathcal{Q}_{n+1}$.


### The absolute value:
$$\lvert{x_i}\rvert=\sqrt{x_i^2}=\rVert{x_i}\lVert_2$$
The sum of absolute values (e.g. the $1$-norm) is replaced by $n$ cones of dimension $2$, e.g.
$[t_i, x_i] \in \mathcal{Q}_2$. Now
$$\Sigma\,t_i \leq 1 + 2c$$ implies $$\Sigma\,\lvert x_i\rvert \leq 1 + 2c$$

\begin{align}\mathbf{x}^{*}=\arg\max_{\mathbf{x} \in \mathbb{R}^n}& \mu^{T}\mathbf{x}\\
\text{s.t. } &\Sigma\,x_i=1\\
             &\Sigma\, t_i \leq 1 + 2c\\
             &[y, G^T x] \in \mathcal{Q}_{n+1}\\
             &y \leq \sigma_{\max}\\
             &[t_i, x_i] \in \mathcal{Q}_2,\,\,i \in [1,\ldots,n]
\end{align}


In [10]:
import cvxpy as cvx
import numpy as np
# make some random data, e.g. cov-matrix and expected returns
n = 100
c = 0.9
C = c * np.ones((n, n)) + (1 - c) * np.eye(n)
mu = 0.05 * np.sin(range(0, n))
# maximal volatility and leverage...
sigma_max = 1.0
excess = 0.3

x = cvx.Variable(n)
constraints = [cvx.sum_entries(x)==1, cvx.norm(x,1)<=1+2*excess, cvx.quad_form(x,C)<=sigma_max*sigma_max]
objective = cvx.Maximize(x.T*mu)
cvx.Problem(objective=objective, constraints=constraints).solve()
f = np.array([x[i].value for i in range(0,n)])
print(f)

print("Sum of positive weights: {0}".format(np.sum(f[f > 0])))
print("Sum of negative weights: {0}".format(np.sum(f[f < 0])))
print("Sum of all weights:      {0}".format(np.sum(f)))

[ -2.08474676e-12   2.06975739e-09   3.86980929e-09   9.67015740e-11
  -1.18755098e-09  -7.96317237e-09  -2.08238157e-10   8.08097931e-10
   3.33631629e-08   3.41228066e-10  -5.24358269e-10  -2.70697656e-01
  -5.11500064e-10   3.50889625e-10   3.77680176e-08   7.87252256e-10
  -2.15578766e-10  -8.52300777e-09  -1.15527228e-09   1.03126870e-10
   4.04874129e-09   2.00004295e-09  -8.13823052e-12  -1.97963310e-09
  -3.31508831e-09  -9.39825101e-11   1.28891591e-09   8.48888404e-09
   1.99166129e-10  -8.00091698e-10  -3.65231770e-08  -3.29649062e-10
   5.49399219e-10   8.51076159e-01   5.08698034e-10  -3.57564090e-10
  -6.02217685e-08  -7.41888004e-10   2.21734012e-10   1.02713237e-08
   1.18216300e-09  -1.13030270e-10  -3.76894909e-09  -1.79605915e-09
   1.00311884e-11   2.21970884e-09   3.54425684e-09   8.39756115e-11
  -1.25568475e-09  -7.00590432e-09  -1.93812365e-10   8.51728907e-10
   2.67631207e-08   3.22423542e-10  -5.50986955e-10  -2.93013765e-02
  -4.86640422e-10   3.70764463e-10

In [2]:
from mosek.fusion import Model, Matrix, Domain, Expr, ObjectiveSense
import numpy as np
# make some random data, e.g. cov-matrix and expected returns
n = 100
c = 0.9
C = c * np.ones((n, n)) + (1 - c) * np.eye(n)
mu = 0.05 * np.sin(range(0, n))
# maximal volatility and leverage...
sigma_max = 1.0
excess = 0.3

def __abs(model, v):
    t = model.variable(int(v.size()), Domain.greaterThan(0.0))
    model.constraint(Expr.hstack(t, v), Domain.inQCone(int(v.size()), 2))
    return t


def __two_norm(model, v):
    t = model.variable(1, Domain.greaterThan(0.0))
    model.constraint(Expr.vstack(t, v), Domain.inQCone())
    return t

    
def __one_norm(model, v):
    return Expr.sum(__abs(model, v))


with Model('equity') as model:
    w = model.variable("w", mu.size, Domain.unbounded())
    
    # sum w_i = 1
    model.constraint(Expr.sum(w), Domain.equalsTo(1.0))

    # sum abs(w_i) <= 1 + 2*excess
    model.constraint(__one_norm(model, w), Domain.lessThan(1.0 + 2 * excess))

    # Norm(G^T*w) <= sigma_max 
    y = __two_norm(model, Expr.mul(Matrix.dense(np.transpose(np.linalg.cholesky(C))), w))
    model.constraint(y, Domain.lessThan(sigma_max))

    # maximize objective mu'w
    model.objective(ObjectiveSense.Maximize, Expr.dot(mu, w)), model.solve()
    f = np.array(w.level())

print("Sum of positive weights: {0}".format(np.sum(f[f > 0])))
print("Sum of negative weights: {0}".format(np.sum(f[f < 0])))
print("Sum of all weights:      {0}".format(np.sum(f)))

Sum of positive weights: 1.300000041390767
Sum of negative weights: -0.30000004711145856
Sum of all weights:      0.9999999942793083


### Summary

- Leverage is here a constraint for the $1$-norm of the weight vector.


- We replace the $1$-norm of a vector of length $n$ with $n$ cones each of dimension $2$.


- We compute the Cholesky decomposition of a $n \times n$ covariance matrix to introduce a cone of dimension $n + 1$.